In [1]:
import firedrake
import matplotlib.pyplot as plt
import icepack
import icepack.plot
import numpy as np

# Firn modeling
Ice-sheet mass-balance estimates derived from surface altimetry observations require careful consideration of spatial variations in firn density. Firn models remain a large source of uncertainty within continental mass-balance estimates because of simplifying assumptions for densification physics, which often neglect the effects of horizontal strain on the integrated firn-air-content (FAC). We aim to include the effects of horizontal divergence and convergence on the densification of snow and firn in continental/catchment scale simulations of ice sheet mass change by building an extensible firn model that ingests multi-demensional strain regime information. The variables that define the state of the firn are the density, the vertical velocity of the firn, and the energy density. The densification of snow to ice is often defined by the material derivative, which in 1D can be written as

\begin{equation*} 
\frac{d\rho}{dt} = \frac{\partial\rho}{\partial t} + w \frac{\partial \rho}{\partial z}
\end{equation*}

The weak form of the densification equation can be written as

\begin{equation*} 
f_\rho = \int_\Omega \frac{\partial\rho}{\partial t} \phi d\Omega + \int_\Omega w \nabla \rho \phi d\Omega - \int_\Omega \frac{d\rho}{dt}\phi d\Omega
\end{equation*}

where we have integrated the density equation over the entire domain, and multiplied by the test function $\phi$. The vertical velocity $w$ can be defined by integrating the densification rate and similarly multiplying by the test function $\eta$.

\begin{equation*} 
w(z,t)=\int \frac{1}{\rho}\cdot \frac{d\rho}{dt}dz
\end{equation*} 
\begin{equation*} 
\frac{\partial w}{\partial z} = \frac{1}{\rho}\cdot \frac{d \rho}{dt}
\end{equation*} 

\begin{equation*} 
f_w = - \int_\Omega \rho \nabla w \psi d\Omega + \int_\Omega  \frac{d \rho}{dt}\psi d\Omega
\end{equation*} 

Following Arshwandan, we can also recast the shallow enthalpy equations into equations for the energy density. These can be broken up into energy density sources, and change as near surface heat conducts through the densifying firn. There are variables like the water/snow fraction that we might also be able to link to radiometric measurements that are regularly made with insar satellite constellations. The weak form for the energy density can be written as 

\begin{equation*} 
f_E = \int_\Omega w \rho \nabla E \psi d\Omega - \int_\Omega 
\left\{ \begin{array}{c}
K_i \\
K_0
\end{array}
\right\}
\nabla E \nabla \psi d\Omega - \int_\Omega \rho \frac{\partial E}{\partial t} \psi d\Omega
\end{equation*} 


Adding these terms we seek test functions that minimize the residual:

\begin{equation*} 
f= f_w + f_\rho + f_E
\end{equation*} 




In [17]:
h_f=firedrake.Constant(200.0)
Lx, Ly = 50e3, 12e3
nx, ny = 1, 1
mesh2d = firedrake.RectangleMesh(nx, ny, Lx, Ly)
mesh = firedrake.ExtrudedMesh(mesh2d, layers=1)

In [18]:
Q = firedrake.FunctionSpace(
    mesh, family='CG', degree=1,
    vfamily='DG', vdegree=10
)

In [19]:
ζ = firedrake.SpatialCoordinate(mesh)

In [20]:
from icepack.constants import year as year
ρ_s = firedrake.Constant(360.0/ year**2 * 1.0e-6) # initial density at the surface
a = firedrake.Constant(.3)
w = firedrake.interpolate(a*(918.0/ year**2 * 1.0e-6)/ρ_s,Q)
ρ = firedrake.interpolate(ρ_s,Q)
Tavg = firedrake.Constant(273.15-15.) # average temperature


In [21]:
model = icepack.models.FirnModel()
opts = {'default_solver_type': 'petsc',
        'default_solver_parameters': {
        'snes_type': 'newtontr',
        'pc_type': 'lu',
        'pc_factor_mat_solver_type': 'mumps'
    }
}
solver = icepack.solvers.FirnSolver(model, **opts)

In [22]:
final_time = 2.0
timestep = 0.5
num_steps = int(final_time / timestep)
dt = final_time / num_steps

for step in range(num_steps):
    ρ = solver.solve_density(dt,
                            firn_thickness=h_f,
                            density=ρ,
                            firn_velocity=w,
                            accumulation=a,
                            temperature=Tavg,
                            surface_density = ρ_s)
    w = solver.solve_velocity(dt,
                            firn_thickness=h_f,
                            density=ρ,
                            firn_velocity=w,
                            accumulation=a,
                            temperature=Tavg,
                            surface_density = ρ_s)
    